In [99]:
# Import packages 
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
# This next command is specifically for Jupyter Notebook
%matplotlib notebook

import requests
from IPython.display import Image

In [100]:
#Loading the data and variable assumptions
data = np.loadtxt('MacroSeries.txt', delimiter = ',')

#We rename the each of the columns of the data with variables corresponding to the model
#(c:consumption, k:capital, w:wage, r=interest rate)
c = data[:,0]
k = data[:,1]
w = data[:,2]
r = data[:,3]

#Parameter assumption
beta = 0.99

In [101]:
#We back the value for "z" using eqn (4). This is the same that was done in PS2
def z_back(r, k, alpha):

    z = np.log(r/(((k)**(alpha-1))*(alpha)))
    
    return z

In [102]:
#We define the moments of the data (the expectations become average over the arguments)
def data_moments(params, args):

    alpha, rho, mu = params
    r, k, c, w = args
        
    z = z_back(r, k, alpha)
    
    #We take the second observation onwards to create the variable en t+1 (_f denotes foward)
    z_f = z[1:]  
    k_f = k[1:]  
    c_f = c[1:]  
    
    # We create the variables z, r, k, c, w adjusted (leaving out the last observation because there is no t+1 for)
    #that observation) so que can make the appropiate operations with z, k and c forward
    z_adj = z[:-1]
    r_adj = r[:-1]
    k_adj = k[:-1] 
    c_adj = c[:-1]
    w_adj = w[:-1]
        
    moment_1 = z_f - rho*z_adj - (1 - rho)*mu
    moment_2 = (z_f - rho*z_adj - (1 - rho)*mu)*z_adj
    moment_3 = (beta*alpha*np.exp(z_f)*k_f**(alpha - 1)*c_adj/c_f) - 1
    moment_4 = ((beta*alpha*np.exp(z_f)*k_f**(alpha - 1)*c_adj/c_f) - 1)*w_adj
        
    mom_1_data = np.mean(moment_1)
    mom_2_data = np.mean(moment_2)
    mom_3_data = np.mean(moment_3)
    mom_4_data = np.mean(moment_4)
    
    return mom_1_data, mom_2_data, mom_3_data, mom_4_data

In [103]:
#We define the moments of the model. In this case is easy beacause all of them are zero (the expecations are equal to zero
#acording to the model presented). Since we have four moments, we create an array with four entries equal to zero.

moms_model = np.array([0, 0, 0, 0])

In [104]:
#We define the vector of error (data_moments - model_moments)
def err_vec(params, args, simple):
    
    mom_1_data, mom_2_data, mom_3_data, mom_4_data = data_moments(params, args)
    moms_data = np.array([mom_1_data, mom_2_data, mom_3_data, mom_4_data])
    moms_model = np.array([0, 0, 0, 0])
        
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data)/moms_data
    
    return err_vec

In [105]:
#We define the criterion that will be use in the minimization problem
def criterion(params, *args):
    
    alpha, rho, mu = params
    r, k, c, w, W = args 
    
    err = err_vec(params, args[:-1], simple=True)
    crit_val = err.T @ W @ err
    
    return crit_val

In [106]:
# We guess some initial values for the parameters (I am taking the values that I got from the MLE estimation of PS2)
alpha_init = 0.7018970334618823
rho_init = 0.4800169691930428
mu_init = 5.077378123134153
params_init = np.array([alpha_init, rho_init, mu_init])

W_hat = np.eye(4)
gmm_args = (r, k, c, w, W_hat)

# GMM estimation 
results = opt.minimize(criterion, params_init, args = (gmm_args), tol = 1e-14,
                           bounds=((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None)), method = 'L-BFGS-B')
    
alpha_GMM, rho_GMM, mu_GMM = results.x

In [107]:
#We print the results for the estimations and the value of the minimized criteria function
print('alpha_GMM=', alpha_GMM)
print('rho_GMM=', rho_GMM)
print('mu_GMM=', mu_GMM)
print('Criterion Function =', results.fun)

alpha_GMM= 0.7019630309421323
rho_GMM= 0.4800169768412977
mu_GMM= 5.077381951948253
Criterion Function = 6.514775672558655e-15
